In [ ]:
import pandas as pd

# Load datasets
customers = pd.read_csv("Customers.csv")
transactions = pd.read_csv("Transactions.csv")
products = pd.read_csv("Products.csv")

# Merge transactions with products to get product details
merged_data = transactions.merge(products, on="ProductID", how="left")

# Merge with customers to get customer details
merged_data = merged_data.merge(customers, on="CustomerID", how="left")

# Preview data
print(merged_data.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  Price_x                      ProductName     Category  Price_y  \
0      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
1      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
2      300.68   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
3      601.36   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   
4      902.04   300.68  ComfortLiving Bluetooth Speaker  Electronics   300.68   

      CustomerName         Region  SignupDate  
0   Andrea Jenkins         Europe  202

In [ ]:
# Total spending per customer
customer_spending = merged_data.groupby("CustomerID")["TotalValue"].sum().rename("TotalSpending")

# Purchase frequency
purchase_frequency = merged_data.groupby("CustomerID")["TransactionID"].nunique().rename("PurchaseFrequency")

# Most purchased category per customer
top_category = merged_data.groupby(["CustomerID", "Category"])["Quantity"].sum().reset_index()
top_category = top_category.loc[top_category.groupby("CustomerID")["Quantity"].idxmax(), ["CustomerID", "Category"]].set_index("CustomerID")

# Average spending per transaction
avg_spending = (customer_spending / purchase_frequency).rename("AvgSpending")

# Merge all features into a single DataFrame
customer_features = pd.concat([customer_spending, purchase_frequency, avg_spending, top_category], axis=1).reset_index()

print(customer_features.head())


  CustomerID  TotalSpending  PurchaseFrequency  AvgSpending     Category
0      C0001        3354.52                  5      670.904  Electronics
1      C0002        1862.74                  4      465.685   Home Decor
2      C0003        2725.38                  4      681.345   Home Decor
3      C0004        5354.88                  8      669.360   Home Decor
4      C0005        2034.24                  3      678.080  Electronics


In [ ]:
customer_features = pd.get_dummies(customer_features, columns=["Category"])
print(customer_features.head())


  CustomerID  TotalSpending  PurchaseFrequency  AvgSpending  Category_Books  \
0      C0001        3354.52                  5      670.904           False   
1      C0002        1862.74                  4      465.685           False   
2      C0003        2725.38                  4      681.345           False   
3      C0004        5354.88                  8      669.360           False   
4      C0005        2034.24                  3      678.080           False   

   Category_Clothing  Category_Electronics  Category_Home Decor  
0              False                  True                False  
1              False                 False                 True  
2              False                 False                 True  
3              False                 False                 True  
4              False                  True                False  


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Drop CustomerID column before similarity calculation
feature_matrix = customer_features.drop("CustomerID", axis=1)

# Compute similarity matrix
similarity_matrix = cosine_similarity(feature_matrix)

# Convert to DataFrame for easier access
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features["CustomerID"], columns=customer_features["CustomerID"])

# Preview similarity matrix
print(similarity_df.head())


CustomerID     C0001     C0002     C0003     C0004     C0005     C0006  \
CustomerID                                                               
C0001       1.000000  0.998868  0.998868  0.997334  0.992278  0.998868   
C0002       0.998868  1.000000  1.000000  0.992734  0.997054  0.999999   
C0003       0.998868  1.000000  1.000000  0.992734  0.997054  1.000000   
C0004       0.997334  0.992734  0.992734  1.000000  0.980581  0.992734   
C0005       0.992278  0.997054  0.997054  0.980581  1.000000  0.997054   

CustomerID     C0007     C0008     C0009     C0010  ...     C0191     C0192  \
CustomerID                                          ...                       
C0001       0.992278  0.995228  0.992276  0.998868  ...  1.000000  0.998868   
C0002       0.997054  0.989461  0.997053  1.000000  ...  0.998868  1.000000   
C0003       0.997054  0.989461  0.997052  0.999999  ...  0.998868  1.000000   
C0004       0.980581  0.999695  0.980579  0.992733  ...  0.997334  0.992734   
C0005  

In [ ]:
# Get the first 20 customers
first_20_customers = customer_features["CustomerID"][:20]

# Create lookalike dictionary
lookalike_dict = {}

for cust_id in first_20_customers:
    # Get the top 3 most similar customers (excluding the customer itself)
    similar_customers = similarity_df[cust_id].sort_values(ascending=False).iloc[1:4]

    # Store in dictionary
    lookalike_dict[cust_id] = list(zip(similar_customers.index, similar_customers.values))

# Convert dictionary to DataFrame
lookalike_df = pd.DataFrame(lookalike_dict.items(), columns=["CustomerID", "Lookalikes"])
print(lookalike_df.head())


  CustomerID                                         Lookalikes
0      C0001  [(C0072, 0.9999999844471962), (C0190, 0.999999...
1      C0002  [(C0199, 0.9999999920068625), (C0025, 0.999999...
2      C0003  [(C0178, 0.9999999999597963), (C0133, 0.999999...
3      C0004  [(C0113, 0.9999999858020151), (C0021, 0.999999...
4      C0005  [(C0197, 0.999999996740546), (C0120, 0.9999999...


In [ ]:
# Save as Lookalike.csv
lookalike_df.to_csv("Lookalike.csv", index=False)
print("Lookalike.csv has been created successfully!")


Lookalike.csv has been created successfully!
